In [1]:
import pandas as pd
from geopy.distance import great_circle
import folium
from folium.plugins import MarkerCluster
from IPython.display import display, HTML
import hashlib

# Expanded dataset with multiple categories of workers, with broader locations around Manila
data = {
    'Provider': [
        'Plumber A', 'Plumber B', 'Plumber C', 'Plumber D', 'Plumber E',
        'Electrician A', 'Electrician B', 'Electrician C', 'Electrician D', 'Electrician E',
        'Painter A', 'Painter B', 'Painter C', 'Painter D', 'Painter E'
    ],
    'Category': [
        'Plumber', 'Plumber', 'Plumber', 'Plumber', 'Plumber',
        'Electrician', 'Electrician', 'Electrician', 'Electrician', 'Electrician',
        'Painter', 'Painter', 'Painter', 'Painter', 'Painter'
    ],
    'Latitude': [
        14.5995, 14.5820, 14.5750, 14.6155, 14.5900,
        14.6090, 14.6125, 14.5678, 14.6200, 14.6050,
        14.6102, 14.5903, 14.5828, 14.5980, 14.6005
    ],
    'Longitude': [
        120.9842, 120.9822, 120.9735, 120.9910, 120.9750,
        120.9900, 120.9882, 120.9701, 120.9820, 120.9805,
        120.9815, 120.9763, 120.9847, 120.9828, 120.9780
    ],
    'Price_per_hour': [50, 55, 60, 65, 70,  # Prices for Plumbers
                       70, 75, 80, 85, 90,   # Prices for Electricians
                       40, 45, 50, 55, 60],  # Prices for Painters
    'Rating': [4.5, 4.1, 4.3, 4.6, 4.8,   # Ratings for Plumbers
               4.0, 3.9, 4.2, 4.5, 4.3,   # Ratings for Electricians
               4.7, 4.4, 4.6, 4.8, 4.9],  # Ratings for Painters
    'Feedback_count': [10, 15, 20, 5, 30,    # Feedback for Plumbers
                       25, 20, 10, 5, 35,    # Feedback for Electricians
                       40, 25, 30, 15, 50],  # Feedback for Painters
    'Certified': [True, True, False, True, False,  # Certifications for Plumbers
                  False, True, True, False, True,   # Certifications for Electricians
                  True, False, True, True, False]    # Certifications for Painters
}

# Create a DataFrame from the data
providers_df = pd.DataFrame(data)

# Function to generate Gravatar URL for each provider
def generate_gravatar_url(provider_name):
    # Create a hash using the provider's name as a unique identifier
    hash_email = hashlib.md5(provider_name.encode()).hexdigest()
    return f"https://www.gravatar.com/avatar/{hash_email}?d=monsterid"

# Assign Gravatar URLs to each provider
providers_df['Image'] = providers_df['Provider'].apply(generate_gravatar_url)

# Function to get user preferences with guided input
def get_user_preferences():
    print("Please enter your preferences for finding a service provider.")
    
    # Select service category
    print("Service Categories: Plumber, Electrician, Painter")
    category = input("Enter your preferred service category (e.g., Plumber): ").capitalize()
    
    # Get budget input
    print("\nEnter your maximum hourly budget for the selected service (e.g., 50, 70):")
    budget = float(input("Maximum budget per hour: "))
    
    # Get location input
    print("\nPlease enter your location coordinates (latitude and longitude) to help us find providers nearby.")
    print("For example, latitude: 14.5995 and longitude: 120.9842")
    location_lat = float(input("Enter your latitude: "))
    location_lon = float(input("Enter your longitude: "))
    
    # Get priority weights
    print("\nWe will score providers based on your budget, proximity, and ratings. Please provide weights for each (0-1), making sure they add up to 1.")
    budget_weight = float(input("Enter weight for budget (0-1): "))
    distance_weight = float(input("Enter weight for distance (0-1): "))
    rating_weight = float(input("Enter weight for rating (0-1): "))
    
    # Ensure weights sum to 1
    total_weight = budget_weight + distance_weight + rating_weight
    if total_weight != 1:
        print("Weights must sum to 1. Adjusting weights proportionally...")
        budget_weight /= total_weight
        distance_weight /= total_weight
        rating_weight /= total_weight
    
    return {
        'Category': category,
        'Budget': budget,
        'Location': (location_lat, location_lon),
        'Priority': {
            'Budget_weight': budget_weight,
            'Distance_weight': distance_weight,
            'Rating_weight': rating_weight
        }
    }

# Filter providers based on the selected category and calculate scores
def score_providers(user_location, providers_df, preferences):
    providers_df = providers_df[providers_df['Category'] == preferences['Category']]
    
    scored_providers = []

    for index, row in providers_df.iterrows():
        distance = great_circle(user_location, (row['Latitude'], row['Longitude'])).kilometers
        budget_score = max(0, (preferences['Budget'] - row['Price_per_hour'])) / preferences['Budget']
        distance_score = max(0, (1 - (distance / 10))) if distance <= 10 else 0
        rating_score = row['Rating'] / 5
        
        total_score = (
            (budget_score * preferences['Priority']['Budget_weight']) +
            (distance_score * preferences['Priority']['Distance_weight']) +
            (rating_score * preferences['Priority']['Rating_weight'])
        )
        
        scored_providers.append({
            "Provider": row['Provider'],
            "Category": row['Category'],
            "Price_per_hour": row['Price_per_hour'],
            "Rating": row['Rating'],
            "Score": round(total_score, 2),
            "Distance": round(distance, 2),
            "Image": row['Image']
        })
    
    scored_providers = sorted(scored_providers, key=lambda x: x['Score'], reverse=True)
    
    display_providers(scored_providers)
    return scored_providers

# Function to display providers with images
def display_providers(providers):
    html_content = "<h2>Matched Providers</h2><table>"
    html_content += "<tr><th>Image</th><th>Provider</th><th>Score</th><th>Distance (km)</th><th>Rating</th><th>Price per hour</th></tr>"
    
    for provider in providers:
        html_content += f"""
            <tr>
                <td><img src="{provider['Image']}" width="50" height="50"></td>
                <td>{provider['Provider']}</td>
                <td>{provider['Score']}</td>
                <td>{provider['Distance']} km</td>
                <td>{provider['Rating']}</td>
                <td>${provider['Price_per_hour']}</td>
            </tr>
        """
    
    html_content += "</table>"
    display(HTML(html_content))

# Plot provider locations based on the matched providers and user location using folium
def plot_provider_locations_folium(matched_providers, user_location):
    m = folium.Map(location=user_location, zoom_start=13)

    folium.Marker(
        location=user_location,
        popup="Your Location",
        icon=folium.Icon(color='blue', icon='person-pin', prefix='fa')
    ).add_to(m)

    marker_cluster = MarkerCluster().add_to(m)

    for provider in matched_providers:
        provider_name = provider['Provider']
        provider_score = provider['Score']
        provider_lat = providers_df[providers_df['Provider'] == provider_name].iloc[0]['Latitude']
        provider_lon = providers_df[providers_df['Provider'] == provider_name].iloc[0]['Longitude']
        
        popup_text = f"<b>{provider_name}</b><br>Score: {provider_score}<br>Distance: {provider['Distance']} km"
        
        folium.Marker(
            location=(provider_lat, provider_lon),
            popup=popup_text,
            icon=folium.Icon(color='green', icon='wrench', prefix='fa')
        ).add_to(marker_cluster)

    display(m)

# Get user preferences through input
user_preferences = get_user_preferences()
matched_providers = score_providers(user_preferences['Location'], providers_df, user_preferences)
plot_provider_locations_folium(matched_providers, user_preferences['Location'])


Please enter your preferences for finding a service provider.
Service Categories: Plumber, Electrician, Painter


Enter your preferred service category (e.g., Plumber):  Electrician



Enter your maximum hourly budget for the selected service (e.g., 50, 70):


Maximum budget per hour:  60



Please enter your location coordinates (latitude and longitude) to help us find providers nearby.
For example, latitude: 14.5995 and longitude: 120.9842


Enter your latitude:  14.5992
Enter your longitude:  120.9882



We will score providers based on your budget, proximity, and ratings. Please provide weights for each (0-1), making sure they add up to 1.


Enter weight for budget (0-1):  0.8
Enter weight for distance (0-1):  0.4
Enter weight for rating (0-1):  0.2


Weights must sum to 1. Adjusting weights proportionally...


Image,Provider,Score,Distance (km),Rating,Price per hour
,Electrician E,0.38,1.05 km,4.3,$90
,Electrician A,0.37,1.11 km,4.0,$70
,Electrician B,0.35,1.48 km,3.9,$75
,Electrician D,0.35,2.41 km,4.5,$85
,Electrician C,0.29,4.0 km,4.2,$80
